In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [15]:
df = pd.read_csv('data/single_step_df_ints_2022-04-14_encoded.csv', header=0)

In [16]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,h.already_seen,y.already_seen,e.already_seen,v.already_seen,o.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,d,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,d,i,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,i,k,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,k,x,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [18]:
df['target'].unique()
df['target'].value_counts()

k    279
p    249
f    172
g    172
d    169
c    152
s    146
b    114
i    107
w     74
a     74
o     73
r     60
u     56
q     43
h     36
v     30
e     22
t     22
z     21
m     13
n      9
x      7
y      1
Name: target, dtype: int64

In [19]:
index_to_drop = df['target'].loc[df['target'] == 'y'].index[0]
index_to_drop

1972

In [21]:
# remove row with target that only occurs once in whole dataset

df_new = df.drop(index=index_to_drop, axis=0)
df_new.reset_index(inplace=True, drop=True)

In [20]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [24]:
df_new[1463:]['target'].value_counts()

d    138
k    116
i    105
p     61
u     56
q     43
b     32
v     30
f     15
r     13
s      8
g      8
x      5
c      4
a      2
h      1
Name: target, dtype: int64

In [26]:
index_to_drop_cooking = df_new[1463:]['target'].loc[df_new[1463:]['target'] == 'h'].index[0]
index_to_drop_cooking

df_cooking = df_new.drop(index=index_to_drop_cooking, axis=0)
df_cooking.reset_index(inplace=True, drop=True)

In [36]:
df_cooking[1461:]

,input,target,start_coords.x,start_coords.y,start_coords.z,h.already_seen,y.already_seen,e.already_seen,v.already_seen,o.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
1461,<start>,i,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1462,i,k,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1463,k,d,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1464,d,p,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1465,<start>,d,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,<start>,d,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2095,d,i,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2096,i,k,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2097,k,x,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [37]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_cooking[1461:], df_cooking[1461:]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [38]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [39]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_cooking['target'].unique()

In [40]:
len(labels)

23

In [41]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [42]:
batch_size = 128

In [43]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_13475/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}
2022-05-23 07:52:46.710502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:52:46.887573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:52:46.887914: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 07:52:46.893179: I tensorflow/core/platform/c

In [44]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [45]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [46]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [47]:
all_inputs, encoded_features = create_input_data(df)

In [48]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [49]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [50]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [51]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
4/4 [==============================] - 9s 953ms/step - loss: 3.0392 - accuracy: 0.1009 - val_loss: 2.4394 - val_accuracy: 0.1979
Epoch 2/100
4/4 [==============================] - 0s 70ms/step - loss: 2.4426 - accuracy: 0.1951 - val_loss: 2.2430 - val_accuracy: 0.2083
Epoch 3/100
4/4 [==============================] - 0s 66ms/step - loss: 2.2809 - accuracy: 0.2377 - val_loss: 2.1473 - val_accuracy: 0.1875
Epoch 4/100
4/4 [==============================] - 0s 64ms/step - loss: 2.1605 - accuracy: 0.2466 - val_loss: 2.0897 - val_accuracy: 0.2396
Epoch 5/100
4/4 [==============================] - 0s 66ms/step - loss: 2.1405 - accuracy: 0.2511 - val_loss: 2.0542 - val_accuracy: 0.2396
Epoch 6/100
4/4 [==============================] - 0s 70ms/step - loss: 2.0837 - accuracy: 0.2780 - val_loss: 2.0060 - val_accuracy: 0.2188
Epoch 7/100
4/4 [==============================] - 0s 70ms/step - loss: 2.0066 - accuracy: 0.2915 - val_loss: 1.9904 - val_accuracy: 0.1979
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 65ms/step - loss: 1.0946 - accuracy: 0.6076 - val_loss: 1.5696 - val_accuracy: 0.4583
Epoch 60/100
4/4 [==============================] - 0s 66ms/step - loss: 1.0923 - accuracy: 0.6121 - val_loss: 1.5748 - val_accuracy: 0.4375
Epoch 61/100
4/4 [==============================] - 0s 68ms/step - loss: 1.0816 - accuracy: 0.6345 - val_loss: 1.5960 - val_accuracy: 0.3958


In [52]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 96ms/step - loss: 1.4806 - accuracy: 0.4896
Accuracy 0.4895833432674408


In [53]:
model.save('models/next_obj_classifier_cooking_2022-05-23')

2022-05-23 07:57:14.274843: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_cooking_2022-05-23/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_cooking_2022-05-23/assets


In [55]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_cooking_2022-05-23/')

In [ ]:
# test prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [30]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [56]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [68]:
nr_of_sequences_ts = len(df[:1461][df[:1461]['input'] == '<start>'])
nr_of_sequences_ts

190

In [71]:
errors = get_prequential_error(df[:1461], reloaded_model, labels, nr_of_sequences_ts)

In [72]:
summed_error = [sum(error) for error in errors[:-1]]

In [73]:
np.median(summed_error)

5.0

In [74]:
with open('results/nn_spatialinfo_prequential_summed_fitted_on_cooking_results_ts_2022-05-23.txt', 'w') as file:
    file.write(str(summed_error))

In [75]:
np.mean(summed_error)

4.942105263157894

In [76]:
len(summed_error)

190